### Training an [unsupervised] neural network to implement the discrete Fourier transform

In [16]:
# https://publik.tuwien.ac.at/files/PubDat_171587.pdf

from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import matplotlib.pyplot as plt

# Generate data:
# N signallength

N = 512
batch = 8192

t = np.linspace(0, 10, N, endpoint=True)
xsin = np.sin(t/4)

# Initialise weight vector to train
# Generate random input data and desired output data
sig = np.random.randn(batch, N) + 1j*np.random.randn(batch, N) + np.pi*np.abs(xsin)

$\begin{align*}y_k &= \displaystyle \sum_{n=0}^{N-1} x_n \cdot \exp(-i\frac{2 \pi k}{N}n)\\
\boldsymbol{y} &= \boldsymbol{x} W  \tag{2}
\end{align*}$



In [12]:
from IPython.display import Image
Image('Data/DFT.gif', width=500)

# Expected weights to match
# Compute Fourier transform using the conventional Fast FT method 
F = np.fft.fft(sig, axis=-1)

# First half of inputs/outputs is real part, second half is imaginary part

$\exp(-i \frac{2 \pi k}{N}n) = cos(\frac{2 \pi k}{N}n) - i sin(\frac{2 \pi k}{N}n)$

In [ ]:
X = np.hstack([sig.real, sig.imag])
Y = np.hstack([F.real, F.imag])

# Create model with no hidden layers, same number of outputs as inputs.
# Zero-valued bias needed.  No activation function, since DFT is linear.
model = Sequential([Dense(N*2, input_dim=N*2, use_bias=False)])
# A Sequential model is appropriate for a plain stack of layers 
# where each layer has exactly one input tensor and one output tensor.

# Compile model
# it configures the model for training.
model.compile(loss='mean_squared_error', optimizer='adamax')
# loss='mean_squared_error'

# Fit the model
# It rains the model for a fixed number of epochs (iterations on a dataset).
history =model.fit(X, Y, epochs=128, batch_size=128, verbose=0)

# list all data in history
print(history.history.keys())
# summarize history for loss
plt.title('Model Loss')
plt.semilogy(history.history['loss'])
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.legend(['Mean Squared Error'])
plt.show()

# Confirm that it works
data = np.arange(N) # harmonics [0 N-1]

def ANN_DFT(x):
    if len(x) != N:
        raise ValueError(f'Input must be length {N}')
    pred = model.predict(np.hstack([x.real, x.imag])[np.newaxis])[0]
    result = pred[:N] + 1j*pred[N:]
    return result

ANN = ANN_DFT(data)
#print(np.abs(ANN))

FFT = np.fft.fft(data)
print(f'ANN matches FFT: {np.allclose(ANN, FFT)}')

# Heat map of neuron weights
fig, ax = plt.subplots()
fig.set_size_inches(12, 9)
plt.imshow(model.get_weights()[0], vmin=-1, vmax=1, cmap='coolwarm') 

In [ ]:
# reconstructed signal fft, learned

In [58]:
# Something as basic as a square or a square root operation requires hidden layer networks and a disproportionately large number of nodes to solve accurately
# As a result,  neural networks struggle with even a simpler transforms  like FFT

In [ ]:
# task : bayes estimator 